In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

Lectura de .csv limpios

In [2]:
data = pd.read_csv("../../files/HR_RAW_DATA.csv", index_col = 0)

employees = pd.read_csv("../../files/df_exploracion_employees.csv" , index_col = 0)

experience = pd.read_csv("../../files/df_employee_exp.csv")

surveys = pd.read_csv("../../files/surveys.csv" , index_col = 0)

employment = pd.read_csv("transofrmacion_patri" )

salary = pd.read_csv("../../files/salarydetails_w_floats.csv" , index_col = 0)

hours = pd.read_csv("../../files/df_Salary_SH.csv")

Concatenación de archivos

In [3]:
df_limpio = pd.concat([data['employeenumber'], employees, experience, surveys, employment, salary, hours], axis = 1, ignore_index = False)

Homogeneización de columnas

In [4]:
nuevas_columnas = {'employeenumber' : 'EmployeeID' , 'NUMBERCHILDREN' : 'NumberChildren' , 'NUMCOMPANIESWORKED' : 'NumCompaniesWorked', 'TOTALWORKINGYEARS' : 'TotalWorkingYears' , 'YEARSWITHCURRMANAGER' : 'YearsWithCurrManager' , 'WORKLIFEBALANCE' : 'WorkLifeBalance'}

In [5]:
df_limpio.rename(columns = nuevas_columnas, inplace = True)

Limpieza de duplicados

In [6]:
df_limpio = df_limpio.drop_duplicates(keep="first")

In [7]:
df_limpio = df_limpio.reset_index()
df_limpio.drop("index", axis=1, inplace=True)

Revisión de aquellos empleados sin EmployeeID que puedan ser el mismo (coinciden en todas las columnas salvo RemoteWork)

In [8]:
# Saca un DataFrame de los empleados con EmployeeID nulo y que coinciden en todo salvo en RemoteWork

# Filtra las filas con valores nulos en 'EmployeeID'
filas_nulas = df_limpio['EmployeeID'].isnull()
df_filas_nulas = df_limpio[filas_nulas]

# Excluye la columna 'RemoteWork' para buscar duplicados
df_sin_remote = df_filas_nulas.drop(columns=['RemoteWork'])

# Busca duplicados en las filas filtradas
duplicados_sin_RW = df_sin_remote[df_sin_remote.duplicated(keep=False)]

# Extrae los indices
indices_duplicados = duplicados_sin_RW.index

# Crea una lista de los indices con las coincidencias
lista_indices_mismo_empleado = df_limpio.loc[indices_duplicados , :].sort_values(['Age' , 'Gender']).index

In [13]:
lista_indices_mismo_empleado

Int64Index([ 312, 1527,  356, 1551,  322, 1532,  364, 1558,  885, 1506,  307,
            1522,  313, 1528,   86, 1473,  325, 1534,  888, 1508, 1009, 1486],
           dtype='int64')

Imputación general de nulos para la columna EmployeeID

In [9]:
def asignar_id(valor, contador):
    
    if pd.isnull(valor):
        contador[0] += 1  # Incrementar el contador si el valor es NaN
        id_nulo = f'nan{contador[0]}'  # Devolver el ID con el contador actualizado
        return id_nulo
    
    else:
        return valor.replace(',0' , '')  # Devolver el valor quitándole los decimales

# Reiniciar el contador cada vez que se aplique la función
def aplicar_asignar_id(columna):
    contador = [0]
    return columna.apply(lambda x: asignar_id(x, contador))

# Aplicar la función a la columna 'EmployeeID' y reasignar los resultados
df_limpio['EmployeeID'] = aplicar_asignar_id(df_limpio['EmployeeID'])


Asignación de la misma EmployeeID a los empleados que aparecen en `lista_indices_mismo_empleado`

In [10]:
indice_sublista = 0

for indice in lista_indices_mismo_empleado[1::2]:
    
    dato = lista_indices_mismo_empleado[indice_sublista]
    
    df_limpio['EmployeeID'][indice] = df_limpio['EmployeeID'][dato]
    
    indice_sublista += 2

/var/folders/z0/ptwngvg97wndky1th_f_rz8r0000gp/T/ipykernel_3281/1808907360.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['EmployeeID'][indice] = df_limpio['EmployeeID'][dato]


In [11]:
df_limpio.to_csv("../../files/df_to_SQL.csv")

In [12]:
df_limpio

,EmployeeID,Age,Gender,MaritalStatus,NumberChildren,DateBirth,Over18,Education,EducationField,NumCompaniesWorked,TotalWorkingYears,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,RelationshipSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Attrition,OverTime,DistanceFromHome,RemoteWork,BusinessTravel,TrainingTimesLastYear,Department,JobRole,JobLevel,DailyRate,HourlyRate,MonthlyRate,MonthlyIncome,PercentSalaryHike,StockOptionLevel,Salary,StandardHours
0,162,51,Male,NaN,NaN,1972,Y,3,NaN,7,NaN,20,NaN,15,15,1,3,3,3.0,3,3.0,No,No,6,1,NaN,5,NaN,research director,5,684.0,51.0,6462.0,19537.0,13,0,1000000000,NaN
1,259,52,Male,NaN,NaN,1971,NaN,4,Life Sciences,0,"34,0",33,NaN,11,9,1,3,1,3.0,2,3.0,No,NaN,1,1,NaN,5,NaN,manager,5,699.0,65.0,5678.0,19999.0,14,1,1000000000,NaN
2,319,42,Male,Married,NaN,1981,NaN,2,Technical Degree,1,"22,0",22,NaN,11,15,1,4,4,NaN,3,3.0,No,No,4,1,travel_rarely,3,Research & Development,manager,5,532.0,58.0,4933.0,19232.0,11,0,1000000000,NaN
3,nan1,47,Female,Married,NaN,1976,Y,4,Medical,3,NaN,20,NaN,5,6,1,3,2,NaN,3,3.0,No,NaN,2,0,travel_rarely,2,NaN,research director,4,359.0,82.0,26703.0,17169.0,19,2,1000000000,NaN
4,nan2,46,Female,Divorced,NaN,1977,Y,3,Technical Degree,2,NaN,19,NaN,2,8,1,1,4,3.0,4,3.0,No,No,3,0,NaN,5,NaN,sales executive,4,1319.0,45.0,7739.0,NaN,12,1,1000000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558,nan101,31,Female,NaN,NaN,1992,NaN,3,Medical,0,"10,0",9,NaN,8,5,1,4,3,1.0,3,3.0,No,NaN,12,1,NaN,2,NaN,healthcare representative,2,616.0,41.0,17369.0,NaN,11,2,1000000000,NaN
1559,966,32,Female,Married,NaN,1991,NaN,4,Medical,1,"8,0",8,NaN,6,3,1,1,3,4.0,3,3.0,No,NaN,3,1,NaN,2,NaN,manufacturing director,2,498.0,93.0,13554.0,6725.0,12,1,1000000000,NaN
1560,972,45,Male,NaN,NaN,1978,NaN,2,NaN,1,NaN,9,NaN,0,8,1,2,2,2.0,3,3.0,No,NaN,4,1,non-travel,5,NaN,laboratory technician,2,805.0,57.0,23163.0,4447.0,12,0,1000000000,NaN
1561,nan411,39,Male,Single,NaN,1984,NaN,5,NaN,0,"9,0",8,NaN,0,7,1,3,4,3.0,4,3.0,No,No,13,1,travel_rarely,3,NaN,sales executive,3,903.0,41.0,2560.0,NaN,18,0,1000000000,NaN
